In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import math
import matplotlib.pyplot as plt
import numpy as np

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    __R = 100@u_kΩ;
    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__name__ = name
        self.__R = R
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )
        
    def wiper(self, w) :
        if w == 0 :
            self.R1.resistance = self.__R * 0.0000001
            self.R2.resistance = self.__R * 0.9999999
        elif w == 1 :
            self.R1.resistance = self.__R * 0.9999999
            self.R2.resistance = self.__R * 0.0000001
        else :
            self.R1.resistance = self.__R * w
            self.R2.resistance = self.__R * (1.0-w)


# theory


In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=1.1)

d += (op1 := elm.Opamp().at([5,0]).flip().label('$U_1$'))
d += elm.Line().left().at(op1.in1).length(d.unit/2).label('3', color='Grey')
d += elm.Resistor().down().length(d.unit/2).label('R5\n27k')
d += elm.Ground()

d += elm.Line().length(d.unit/2).left().at(op1.in2).label('1', color='Grey')
d += elm.Dot()
d.push()
d += elm.Line().length(d.unit/2).left()
d += ( Dot0 := elm.Dot())
d += elm.Resistor().label('R2\n27k')
d += elm.Dot()
d.push()
d += elm.Resistor().label('R1\n470k').down()
d += elm.Ground()
d.pop()
d += elm.Line().length(d.unit/2).left()
d += elm.Dot(open=True).label('trigger', loc="lft")
d.pop()
d += elm.Line().up().length(d.unit)
d += ( R3 := elm.Resistor().right().length(d.unit*1.75).label('R3\n470k'))
d += elm.Line().down().toy(op1.out)
d += elm.Dot()
d.push()
d += elm.Line().left().tox(op1.out).label('2', color='Grey')

d.pop()
d += elm.Diode().down().label('D1')
d += elm.Diode().down().label('D2').label('4', loc='start', color='Grey')
d += elm.Dot()
d.push()

d += elm.Resistor().down().label('R9\n380k')
d += ( Dot1 := elm.Dot()).label('5', loc='left', color='Grey')
d += elm.Resistor().down().label('R10\n47k')
d += elm.Ground()

d.pop()
d += elm.Line().right().label('6', color='Grey')
d += (op2 := elm.Opamp().anchor('in2').label('$U_2$'))

d += elm.Line().at(op2.in1).left().length(d.unit/2).label('7', color='Grey')
d += elm.Line().up()
d += elm.Dot()
d.push()
d += elm.Diode().right().length(d.unit*2).label('D5')
d += elm.Dot()

d += elm.Dot()
d += elm.Diode().label('D7')
d += elm.ResistorVarIEEE()
d += elm.Dot()

d.pop()
d += elm.Line().up()
d += ( Dot2 := elm.Dot())
d += elm.Diode().right().reverse().length(d.unit*2).label('D4')
d += elm.Dot()
d.push()
d += elm.Diode().reverse().label('D6')
d += elm.ResistorVarIEEE()

d += elm.Line().down().length(d.unit*2)
d += elm.Dot()
d.push()
d += elm.Capacitor().label('C1')
d += elm.Ground()

d.pop()
d += elm.Line().right().length(d.unit)
d += (op3 := elm.Opamp().anchor('in2').label('$U_3$'))

d.pop()
d += elm.Line().down().toy(op2.out)
d += elm.Line().left().tox(op2.out).label('8', color='Grey')

d += elm.Line().at(op3.in1).length(d.unit/2).left()

d += elm.Line().up().toy(R3.end)
d += elm.Dot()
d.push()
d += elm.Resistor().left().label('R').tox(Dot2.end)
d += elm.Line().down().toy(Dot2.end)

d.pop()
d += elm.Line().right().length(d.unit*2)
d += elm.Line().down().toy(op3.out)
d += elm.Dot()
d.push()

d += ( D3 := elm.Line().left().length(d.unit*0.4))
d += elm.Dot()
d += elm.Line().left().tox(op3.out)

d.pop()
d += elm.Resistor().down()
d += elm.Dot()
d.push()
d += elm.Resistor().down()
d += elm.Ground()

d.pop()
d += elm.Line().right().length(d.unit/2)
d += elm.Dot(open=True).label('out', loc='right')

#d += elm.Line().at(D3.end).down().length(d.unit*3)
#d += elm.Line().left()

d += elm.Line().at(Dot0.end).down().length(d.unit*4)
d += elm.Resistor()
d += elm.Dot()
d.push()
d += elm.Resistor()
d += elm.Ground()

d.pop()
d += elm.Line().right().length(d.unit*3)
d += (op4 := elm.Opamp().anchor('out').label('$U_4$')).left()

d += elm.Line().right().at(op4.in1).tox(D3.end)
d += elm.Line().up().toy(D3.end)

d += elm.Line().right().at(op4.in2)
d += elm.Line().up().toy(Dot1.end)
d += elm.Line().left().tox(Dot1.end)

d.draw()




In [ ]:
circuit = Circuit('ad_ar')
circuit.include(spice_library['TL072'])
circuit.include(spice_library['D1N4148'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'TRIGGER', circuit.gnd, 'DC 0 AC 0 PULSE(0 5 20m 0m 0m 10m 100m)')

#the opamps
#                     NON-INVERTING INPUT
#                     |           INVERTING INPUT
#                     |           |             POSITIVE POWER SUPPLY
#                     |           |             |       NEGATIVE POWER SUPPLY
#                     |           |             |       |       OUTPUT
#                     |           |             |       |       |
circuit.X(1, 'TL072', '1',        '2',          '+15V', '-15V', '3')
circuit.X(2, 'TL072', 'saw',      'X2neg',      '+15V', '-15V', 'X2out')
circuit.X(3, 'TL072', 'position', 'saw',        '+15V', '-15V', 'X3out')
circuit.X(4, 'TL072', 'X4pos',    'X4neg',      '+15V', '-15V', 'X2neg')

#CV voltage divider
circuit.R(1, 'TRIGGER', circuit.gnd, 470@u_kΩ)
circuit.R(2, 'TRIGGER', '1', 27@u_kΩ)
circuit.R(3, '1', '2', 470@u_kΩ)
circuit.R(5, '3', circuit.gnd, 27@u_kΩ)
circuit.R(9, '6', '5', 380@u_kΩ)
circuit.R(10, '5', circuit.gnd, 47@u_kΩ)

circuit.D('1', '2', '4', model='D1N4148')
circuit.D('2', '4', '6', model='D1N4148')


The CD4069UB device consist of six CMOS inverter circuits. These devices are intended for all generalpurpose inverter applications where the mediumpower TTL-drive and logic-level-conversion. This device is not ideal for linear amplification, but can be used as such and will add a lot of soft-clipping. Soft-clipping is expected from a valve. This makes this device special for musical usage and can be found in guitar distortion pedals [[2](http://www.runoffgroove.com/ubescreamer.html)] or the wasp filter [[3](https://www.schmitzbits.de/wasp.html)]. 
{: class="mt-6"}

In [ ]:
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

results = []

#for i in np.arange(0, 5, 1) :
circuit = parser.build_circuit(ground=5)
circuit.include(spice_library['TL074c'])
circuit.include(spice_library['D1N4148'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', '/TRIGGER', circuit.gnd, 'DC 0 AC 0 PULSE(0 5 20m 0m 0m 10m 100m)')

for c in ( VoltageDivider(R=100@u_kΩ, w=0.1, name='POT1'), 
        VoltageDivider(R=500@u_kΩ, w=0.1, name='POT2') ) :
    circuit.subcircuit(c)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.options('INTERP')
#simulator.initial_condition("Net-_C5-Pad1_"=0.0)
analysis = simulator.transient(step_time=1@u_us, start_time=10@u_ms, end_time=200@u_ms)


In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
ax1_buffer.plot(u_ms(analysis['/TRIGGER'].abscissa), analysis['/TRIGGER'], color='Grey')
ax1_buffer.plot(u_ms(analysis['Net-_D2-Pad1_'].abscissa), analysis['Net-_D2-Pad1_'], color='Blue')
#ax1_buffer.plot(u_ms(analysis['Net-_C5-Pad1_'].abscissa), analysis['Net-_D6-Pad2_'], color='Magenta')
ax1_buffer.plot(u_ms(analysis['Net-_D1-Pad1_'].abscissa), analysis['Net-_D5-Pad2_'], color='Green')
ax1_buffer.plot(u_ms(analysis['/OUT'].abscissa), analysis['/OUT'], color='Red')

ax1_buffer.legend(('Vin_a [V]', 'Vin_b', 'Vout [V]'), loc=(0.75,0.8))

plt.tight_layout()
plt.show()

the chip should be powered by +5V/GND. With higher voltage the heat dissipation will be to big and the chip will be damaged. the chip is powered on pin 7 and 14 with +5V and Ground. the other pins can be used as amplifier. 

In [ ]:
import sympy as sp
V1, V2, R1, R2 = sp.symbols("V1 V2 R1 R2")
eq1 = sp.Eq(V2, (V1 * R1) / (R1 + R2))
eq1.rhs.evalf(subs={R1: 2.7@u_kOhm, R2: 22@u_kOhm, V1: 15@u_V})

In [ ]:
kicad_netlist_path = directory_path.joinpath('main', 'pulse delay 1.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

results = []

#for i in np.arange(0, 5, 1) :
circuit = parser.build_circuit(ground=5)
circuit.include(spice_library['TL072c'])
circuit.include(spice_library['UA555'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', '/IN', circuit.gnd, 'DC 0 AC 0 PULSE(0 5 5m 0m 0m 10m)')

for c in ( VoltageDivider(R=1@u_MΩ, w=0.5, name='POT1'), 
        VoltageDivider(R=1@u_MΩ, w=0.5, name='POT2') ) :
    circuit.subcircuit(c)

print(circuit)

#circuit.R(100, '/shape_a', '/shape_b', 25@u_kΩ)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.options('INTERP')
#simulator.initial_condition("Net-_C5-Pad1_"=0.0)
analysis = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=100@u_ms)


fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
#ax1_buffer.plot(u_ms(analysis['/IN'].abscissa), analysis['/IN'], color='Grey')
#ax1_buffer.plot(u_ms(analysis['/del_out'].abscissa), analysis['/del_out'], color='Red')
ax1_buffer.plot(u_ms(analysis['Net-_C4-Pad2_'].abscissa), analysis['Net-_C4-Pad2_'], color='Red')
ax1_buffer.plot(u_ms(analysis['/del_out'].abscissa), analysis['/del_out'], color='Green')
ax1_buffer.plot(u_ms(analysis['Net-_C6-Pad2_'].abscissa), analysis['Net-_C6-Pad2_'], color='Blue')

ax1_buffer.legend(('Vin_a [V]', 'Vin_b', 'Vout [V]'), loc=(0.75,0.8))

plt.tight_layout()
plt.show()

{% include bom.html content="summe-main-bom" %}
{% include callouts.html %}
{% include reports.html%}

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<div class="hero is-medium" style="background: url('{{ '/assets/cmos_buffer_files/tmb_ube_screamer.jpg' | relative_url }}') no-repeat center center; background-size: cover; background-attachment: fixed;">
   <div class="hero-body">
        <div class="content has-text-centered">
        </div>
    </div>
</div>
'''))

# references

https://studylib.net/doc/18188991/the-all-in-ad-ar-envelope-generator

https://electro-music.com/wiki/pmwiki.php?n=Schematics.AD-ARByIanFritz 

https://electro-music.com/forum/topic-21723-25.html 

https://electro-music.com/forum/phpbb-files/fritzdualadar_builddoc_283.pdf 

https://nanopdf.com/download/the-all-in-ad-ar-envelope-generator_pdf

https://www.renesas.com/us/en/document/dst/icm7555-icm7556-datasheet